### Load environment variables

In [1]:
import os
import sys  
sys.path.append("..")  

from dotenv import load_dotenv
_ = load_dotenv('.env')

# This lines should be modified according to your environment variables
os.environ['OPENAI_API_TYPE'] =  os.getenv('OPENAI_API_TYPE')
os.environ['OPENAI_API_VERSION'] = os.getenv("AZURE_OPENAI_API_VERSION")
os.environ['OPENAI_API_BASE'] = os.getenv("AZURE_OPENAI_API_ENDPOINT")
os.environ['OPENAI_API_KEY'] = os.getenv("AZURE_OPENAI_API_KEY")
os.environ['LANGCHAIN_HUB_API_KEY'] = os.getenv("LANG_SMITH_API_KEY")  ## This is not necessary. You can pull without this.
LLM_DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_LLM_DEPLOYMENT_NAME")

### Get a random persona

In [2]:
from user_persona import UserPersona, get_random_user_persona
persona = get_random_user_persona().json()
persona

'{"gender": "Non_binary", "name": "Lauren Espinoza", "language": "English", "location": "Argentina", "age": 81, "hobbies": ["playing_with_electronic_devices", "yoga", "running", "pottery", "birdwatching"], "talkative": true, "characteristics": ["Boisterous", "NotWitty"], "education_level": "unknown"}'

### Fetch the prompt and create a chain (using LCEL)

In [3]:
from langchain import hub
from langchain.chat_models import AzureChatOpenAI

prompt = hub.pull("jet-taekyo-lee/persona-based-dialog")
chat_model = AzureChatOpenAI(deployment_name=LLM_DEPLOYMENT_NAME, temperature=0)

def streaming_output(response_generator):
    for chunk in response_generator:
        print(chunk.content, end='')    

chain = prompt | chat_model

### Get a response with an empty input (Initiating a conversation)

In [4]:
response = chain.stream({'persona':persona, 'input': ''})
streaming_output(response)

## Persona Analysis
- **Characteristics**: Boisterous, NotWitty
- **Other Attributes**: Non-binary, named Lauren Espinoza, speaks English, lives in Argentina, 81 years old, hobbies include playing with electronic devices, yoga, running, pottery, birdwatching, talkative, education level unknown.

## Conflict Detection in persona
- **Conflict 1**: The persona is 81 years old and enjoys running. This could be seen as a conflict as running is a physically demanding activity that may not be typical for someone of this age.
- **Conflict 2**: The persona is boisterous but not witty. These characteristics can sometimes conflict as boisterous individuals are often perceived as being witty.

## Conflict Resolution in persona
- **Adjusted Characteristics**: The persona is 81 years old but is in excellent physical condition, allowing them to enjoy running. The persona is boisterous, meaning they are loud and full of energy, but they are not particularly witty, meaning they may not always say the c

In [5]:
response = chain.stream({'persona':persona, 'input': 'Life is so boring. Why is that?'})
streaming_output(response)

## Persona Analysis
- **Characteristics**: Boisterous, NotWitty
- **Other Attributes**: Non-binary, named Lauren Espinoza, speaks English, lives in Argentina, 81 years old, enjoys playing with electronic devices, yoga, running, pottery, birdwatching, talkative, education level unknown.

## Conflict Detection in persona
- **Conflict 1**: The persona is 81 years old and enjoys running. This might be a bit conflicting as not all 81-year-olds are physically capable of running.
- **Conflict 2**: The persona is boisterous but not witty. These characteristics can sometimes conflict as boisterous individuals are often perceived as witty due to their outgoing nature.

## Conflict Resolution in persona
- **Adjusted Characteristics**: The persona is 81 years old and enjoys brisk walking instead of running. The persona is boisterous but not necessarily witty in a traditional sense, they might be more straightforward and less inclined to use humor.

## Detailed Persona Description
Lauren Espinoza i